<a href="https://colab.research.google.com/github/shashank-r97/NLP/blob/master/Style/SimpleRNN_Glove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

In [0]:
#!pip install -U imbalanced-learn

In [3]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential 
from keras.layers import Input, Masking
from keras.layers import Dense, Input, Reshape
from keras.layers import Dropout, Activation
from keras.layers import Dense, GlobalAveragePooling1D, Activation
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM, SimpleRNN
from keras.layers import Embedding
import keras.backend as K 
from imblearn.combine import SMOTETomek
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.layers.convolutional import Conv1D
from keras.layers.merge import concatenate
from keras.layers.convolutional import MaxPooling1D
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.phrases import Phraser, Phrases

Using TensorFlow backend.


In [4]:
%%time
df = pd.read_csv('/content/drive/Shared drives/DSO 560 NLP Project/data.csv')
df = df[df['attribute_name'] == 'style'].drop(columns = ['attribute_name'])
df['attribute_value'] = df['attribute_value'].apply(lambda x: 'businesscasual' if x == 'business casual' else x)

CPU times: user 408 ms, sys: 75.4 ms, total: 483 ms
Wall time: 513 ms


In [5]:
df.head()

,product_id,brand,product_full_name,description,brand_category,attribute_value,details
0,01DPGV4YRP3Z8J85DASGZ1Y99W,frame,les second medium noir,"minimal , modern styling meet refined luxury l...",accessory,casual,NaN
5,01DPH1DEN9G2WM7WAMJMD0A9W4,j crew,tie waist shirtdress stripe,take classic button silhouette turn ultra flat...,dressesandjumpsuits,casual,NaN
12,01E2KYW52BAG606GQ7A9H5R0KD,alo,interval microfleece pullover hoodie,articulate seam extra wide rib hem create shap...,unknown,casual,"xs 0 2 , s 4 6 , m 8 10 , l 12 14"
13,01DT513RRYT3SKH6X25G5VCH6B,chlo,leather ankle boot,heel measure approximately 55 mm 2 inch 30 mm ...,shoe boots ankle,androgynous,"fit small size , size large normal available s..."
20,01E2KM0KW6NB1JKMZVRXR6H8G2,alo,stadium quarter zip hoodie,supersoft hoodie design elastic hem cuff perfe...,unknown,casual,"xs 0 2 , s 4 6 , m 8 10 , l 12 14"


In [0]:
df['text'] = (df['brand'] + ' ' + df['product_full_name'] + ' ' + df['description'] + ' ' + df['brand_category']).apply(str)

In [0]:
X = df['text'].values
y = pd.get_dummies(df['attribute_value'])
label_list = y.columns
y = y.values

In [0]:
tokenizer = Tokenizer(num_words=5000, oov_token="UNKNOWN_TOKEN")
tokenizer.fit_on_texts(X)

In [0]:
X = tokenizer.texts_to_sequences(X)

In [0]:
length = max([len(s.split()) for s in df['text']])
vocab_size = len(tokenizer.word_index) + 1

In [0]:
X = pad_sequences(X, maxlen=length, padding='post')

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)

In [0]:
resampler = SMOTETomek(sampling_strategy = 'auto')
X_train, y_train = resampler.fit_resample(X_train, y_train)

In [0]:
num_classes = y.shape[1]

In [15]:
def load_glove_vectors():
    embeddings_index = {}
    with open('/content/drive/My Drive/NLP/glove.6B.100d.txt') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings_index[word] = coefs
    print('Loaded %s word vectors.' % len(embeddings_index))
    return embeddings_index


embeddings_index = load_glove_vectors()

Loaded 400000 word vectors.


In [0]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, 100))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: # check that it is an actual word that we have embeddings for
        embedding_matrix[i] = embedding_vector

In [0]:
def define_model():
    
    model = Sequential()
    model.add(Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=length, trainable=False))
    model.add(Masking(mask_value=0.0)) # masking layer, masks any words that don't have an embedding as 0s.
    model.add(SimpleRNN(units=64, activation = 'relu'))#, input_shape=(1, length)))
    model.add(Dense(16, activation = 'relu'))
    model.add(Dense(num_classes, activation='sigmoid'))
    
    # Compile the model
    model.compile(
    optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

def define_model():
    inputs = Input(shape=(length1+length2,))
    #x1 = Conv1D(filters=num_classes, kernel_size=1, padding='valid')(inputs1)
    embedding = Embedding(input_dim=vocab_size1, output_dim=100)(inputs)

    #inputs2 = Input(shape=(length2,))
    #x2 = Conv1D(filters=num_classes, kernel_size=1, padding='valid')(inputs2)
    #dense2 = Dense(100, activation='relu')(inputs2)
    #embedding2 = Embedding(input_dim=vocab_size2, output_dim=100)(inputs2)

    #merged = concatenate([inputs1, inputs1])
    x = LSTM(16, return_sequences=True, dropout=0.2, recurrent_dropout=0.15)(embedding)

    #x = Dense(100, activation = 'relu')(inputs)


    x = Conv1D(filters=num_classes, kernel_size=1, padding='valid')(x)
    x = Conv1D(filters=num_classes, kernel_size=310, padding='valid')(x)
    x = Reshape((num_classes,))(x)
    #x = Dense(num_classes)(x)
    out = Activation('sigmoid')(x)

    model = Model(inputs = [inputs], outputs = out)

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [18]:
model = define_model()
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 155, 100)          611400    
_________________________________________________________________
masking_1 (Masking)          (None, 155, 100)          0         
_________________________________________________________________
simple_rnn_1 (SimpleRNN)     (None, 64)                10560     
_________________________________________________________________
dense_1 (Dense)              (None, 16)                1040      
_________________________________________________________________
dense_2 (Dense)              (None, 11)                187       
Total params: 623,187
Trainable params: 11,787
Non-trainable params: 611,400
_________________________________________________________________


In [19]:
model.fit(X_train, y_train, validation_split=0.2, epochs=10, batch_size=512)

Train on 20376 samples, validate on 5094 samples
Epoch 1/10
20376/20376 [==============================] - 11s 544us/step - loss: 0.4143 - accuracy: 0.8476 - val_loss: 0.4185 - val_accuracy: 0.9090
Epoch 2/10
20376/20376 [==============================] - 10s 501us/step - loss: 0.3067 - accuracy: 0.9091 - val_loss: 0.4326 - val_accuracy: 0.9091
Epoch 3/10
20376/20376 [==============================] - 10s 515us/step - loss: 0.2961 - accuracy: 0.9091 - val_loss: 0.4538 - val_accuracy: 0.9091
Epoch 4/10
20376/20376 [==============================] - 11s 538us/step - loss: 0.2918 - accuracy: 0.9091 - val_loss: 0.4641 - val_accuracy: 0.9091
Epoch 5/10
20376/20376 [==============================] - 10s 505us/step - loss: 0.2882 - accuracy: 0.9091 - val_loss: 0.4707 - val_accuracy: 0.9091
Epoch 6/10
20376/20376 [==============================] - 10s 505us/step - loss: 0.2844 - accuracy: 0.9091 - val_loss: 0.4769 - val_accuracy: 0.9090
Epoch 7/10
20376/20376 [==============================] -

In [20]:
model.evaluate(X_test, y_test)

1089/1089 [==============================] - 1s 654us/step


[0.28955444595045293, 0.909090518951416]

In [0]:
results = model.predict(X)

In [0]:
results_df = pd.DataFrame()
results_mask = results > 0.1
for i in range(len(label_list)):
    results_df[label_list[i]] = results_mask[:,i]
    results_df[label_list[i]] = results_df[label_list[i]].apply(int)

In [23]:
results_df['sum']  = 0
for key in label_list:
    results_df['sum'] = results_df['sum'] + results_df[key]
(results_df['sum'] == 0).sum()

0

In [24]:
results_df = results_df.drop(columns = ['sum'])
for key in label_list:
    results_df[key] = results_df[key].apply(lambda x: key if x == 1 else '')

results_df['attribute_value'] = ''
for key in label_list:
    results_df['attribute_value'] = results_df['attribute_value'] + ' ' + results_df[key]

results_df['attribute_value'] = results_df['attribute_value'].apply(lambda x: ', '.join(x.split()))
results_df.head()

,androgynous,athleisure,boho,businesscasual,casual,classic,edgy,glam,modern,retro,romantic,attribute_value
0,,athleisure,,,,classic,edgy,,,,,"athleisure, classic, edgy"
1,androgynous,athleisure,boho,businesscasual,,classic,edgy,glam,,,,"androgynous, athleisure, boho, businesscasual,..."
2,,,,businesscasual,casual,classic,,,,,,"businesscasual, casual, classic"
3,androgynous,,,businesscasual,,classic,edgy,,modern,,,"androgynous, businesscasual, classic, edgy, mo..."
4,,,,,casual,classic,,,,,,"casual, classic"


In [25]:
results_df['attribute_value'].value_counts()

businesscasual, casual, classic, modern                 1965
businesscasual, casual, classic, edgy, modern           1665
androgynous, businesscasual, casual, edgy               1384
androgynous, businesscasual, casual, classic, modern     427
casual, classic, modern                                  317
                                                        ... 
athleisure, boho, casual, edgy, glam, modern               2
athleisure, classic, edgy, glam                            2
boho, businesscasual, casual, classic, glam                2
athleisure, boho, modern                                   2
athleisure, edgy                                           1
Name: attribute_value, Length: 191, dtype: int64